In [20]:
############################ PRE PROCESSAMANTOOOOOOOOO"""""
from PIL import Image, ImageEnhance
import os

input_dirs = [
    "/kaggle/input/building-extraction-generalization-2024/train",
    "/kaggle/input/building-extraction-generalization-2024/val",
    "/kaggle/input/building-extraction-generalization-2024/test",
]
output_root = "/kaggle/working/processed"
gamma = 0.75
exponent = (1.0 / gamma) if gamma < 1 else gamma
gamma_table_8bit = [int(((i / 255.0) ** exponent) * 255.0) for i in range(256)]
gamma_table_16bit = None

for split_dir in input_dirs:
    if not os.path.exists(split_dir):
        continue
    split_name = os.path.basename(split_dir)
    for root, dirs, files in os.walk(split_dir):
        rel_path = os.path.relpath(root, split_dir)
        output_dir = os.path.join(output_root, split_name, rel_path) if rel_path != "." else os.path.join(output_root, split_name)
        os.makedirs(output_dir, exist_ok=True)
        for filename in files:
            fname_lower = filename.lower()
            if not (fname_lower.endswith(".jpg") or fname_lower.endswith(".jpeg") or 
                    fname_lower.endswith(".png") or fname_lower.endswith(".tif") or fname_lower.endswith(".tiff")):
                continue
            input_path = os.path.join(root, filename)
            output_path = os.path.join(output_dir, filename)
            try:
                with Image.open(input_path) as im:
                    mode = im.mode
                    if mode == "P":
                        im = im.convert("RGB")
                        mode = im.mode
                    if mode == "RGB":
                        r, g, b = im.split()
                        r = r.point(gamma_table_8bit)
                        g = g.point(gamma_table_8bit)
                        b = b.point(gamma_table_8bit)
                        im = Image.merge("RGB", (r, g, b))
                    elif mode == "RGBA":
                        r, g, b, a = im.split()
                        r = r.point(gamma_table_8bit)
                        g = g.point(gamma_table_8bit)
                        b = b.point(gamma_table_8bit)
                        rgb_im = Image.merge("RGB", (r, g, b))
                        rgb_im = ImageEnhance.Contrast(rgb_im).enhance(1.3)
                        r, g, b = rgb_im.split()
                        im = Image.merge("RGBA", (r, g, b, a))
                        im.save(output_path)
                        continue
                    elif mode == "L":
                        im = im.point(gamma_table_8bit)
                    elif mode.startswith("I;16"):
                        if gamma_table_16bit is None:
                            gamma_table_16bit = [int(((i / 65535.0) ** exponent) * 65535.0) for i in range(65536)]
                        im = im.point(gamma_table_16bit, mode)
                    im = ImageEnhance.Contrast(im).enhance(1.3)
                    im.save(output_path)
            except Exception as e:
                print(f"Failed to process {input_path}: {e}")


In [ ]:
# MODELOOOO 
#1. Register COCO-format datasets
# --- Pillow compatibility shim: define aliases Detectron2 expects ---
import json

from PIL import Image as _Image
try:
    from PIL.Image import Resampling as _Resampling
except Exception:
    _Resampling = None

def _alias(dst, src):
    if not hasattr(_Image, dst):
        if hasattr(_Image, src):
            setattr(_Image, dst, getattr(_Image, src))
        elif _Resampling is not None and hasattr(_Resampling, src):
            setattr(_Image, dst, getattr(_Resampling, src))

# Map missing names used by some Detectron2 versions
_alias("LINEAR", "BILINEAR")      # use BILINEAR for "LINEAR"
_alias("CUBIC", "BICUBIC")        # use BICUBIC for "CUBIC"
_alias("ANTIALIAS", "LANCZOS")    # ANTIALIAS → LANCZOS in Pillow>=10

from detectron2.utils.logger import setup_logger
setup_logger()  # call this before creating the trainer


!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git@4a5e6d79e626837a0317195131afaca64b3f4e2d'
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances

TRAIN_IMG_DIR = "/kaggle/working/processed/train/"
VAL_IMG_DIR   = "/kaggle/working/processed/val/"
#TRAIN_IMG_DIR = "/kaggle/input/building-extraction-generalization-2024/train"
#VAL_IMG_DIR   = "/kaggle/input/building-extraction-generalization-2024/val"

TRAIN_JSON   = "/kaggle/input/building-extraction-generalization-2024/train/train.json"
VAL_JSON     = "/kaggle/input/building-extraction-generalization-2024/val/val.json"
def reregister_coco(name, json_file, image_root):
    # If the dataset was registered earlier in this process, clear it first
    if name in DatasetCatalog.list():
        DatasetCatalog.remove(name)
    try:
        MetadataCatalog.remove(name)  # avoid stale metadata
    except KeyError:
        pass
    register_coco_instances(name, {}, json_file, image_root)

reregister_coco("train_dataset", TRAIN_JSON, TRAIN_IMG_DIR)
reregister_coco("val_dataset",   VAL_JSON,   VAL_IMG_DIR)


# 2. Configure Mask R-CNN and train
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train_dataset",)
cfg.DATASETS.TEST  = ("val_dataset",)  # for evaluation
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1      # one class (building)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1892*6 #10 # adjust as needed
# Train the model
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# 3. (Optional) Validate on the val set
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultPredictor
# We can evaluate using Detectron2’s COCOEvaluator (computes AP for instance masks):contentReference[oaicite:4]{index=4}.
evaluator = COCOEvaluator("val_dataset", cfg, False, output_dir="./output")
evaluator.reset()  # makes _coco_api
evaluator._coco_api.dataset.setdefault("info", {})
val_loader = build_detection_test_loader(cfg, "val_dataset")
# Run inference on val and compute metrics
metrics = inference_on_dataset(trainer.model, val_loader, evaluator)
print("Validation metrics:", metrics)


  Cloning https://github.com/facebookresearch/detectron2.git (to revision 4a5e6d79e626837a0317195131afaca64b3f4e2d) to /tmp/pip-req-build-multm04u
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-multm04u
  Running command git rev-parse -q --verify 'sha^4a5e6d79e626837a0317195131afaca64b3f4e2d'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git 4a5e6d79e626837a0317195131afaca64b3f4e2d
  Running command git checkout -q 4a5e6d79e626837a0317195131afaca64b3f4e2d
  Resolved https://github.com/facebookresearch/detectron2.git to commit 4a5e6d79e626837a0317195131afaca64b3f4e2d
  Preparing metadata (setup.py) ... done
[10/07 00:19:20 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, 

In [74]:
#### TEST e .CSV

import os
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# --- Build predictor from your trained weights ---
ckpt_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
if not os.path.exists(ckpt_path):
    with open(os.path.join(cfg.OUTPUT_DIR, "last_checkpoint")) as f:
        ckpt_path = f.read().strip()

cfg_pred = cfg.clone()
cfg_pred.MODEL.WEIGHTS = ckpt_path
cfg_pred.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg_pred)   # <— use cfg_pred, not cfg

# --- Inference on test and write polygons as strings ---
import os, cv2, numpy as np, pandas as pd, ast

TEST_IMG_DIR = "/kaggle/working/processed/test/image"
results = []

test_images = sorted([f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(".tif")])

for img_name in test_images:
    im = cv2.imread(os.path.join(TEST_IMG_DIR, img_name))
    outputs = predictor(im)

    polygons = []   # list of polygons; each polygon = list of (x, y) ints

    if "instances" in outputs and len(outputs["instances"]) > 0:
        masks = outputs["instances"].pred_masks.cpu().numpy()

        for mask in masks:
            mask_uint8 = (mask.astype(np.uint8) * 255)

            # External boundaries only; reduce point count
            contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for cnt in contours:
                if cnt.shape[0] < 3:  # need at least a triangle
                    continue

                # Optional: further simplify if boundaries are too dense
                # epsilon = 0.002 * cv2.arcLength(cnt, True)
                # cnt = cv2.approxPolyDP(cnt, epsilon, True)

                pts = cnt.squeeze(1).tolist()  # [[x,y], [x,y], ...]
                if len(pts) < 3:
                    continue

                # ensure closed by repeating first point
                if pts[0] != pts[-1]:
                    pts.append(pts[0])

                # convert to list of tuples [(x,y), ...] with ints
                poly = [(int(x), int(y)) for x, y in pts]
                polygons.append(poly)

    # IMPORTANT: Coordinates must be a STRING representing list-of-polygons
    # Example for one image: "[[(x0,y0),(x1,y1),...], [(x0,y0),(x1,y1),...]]"
    coord_str = str(polygons) if len(polygons) > 0 else "[]"

    image_id = int(os.path.splitext(img_name)[0])  # "0001.jpg" -> 1
    results.append({"ImageID": image_id, "Coordinates": coord_str})

sub_df = pd.DataFrame(results, columns=["ImageID", "Coordinates"])


# verify that each Coordinates cell can be parsed as a Python literal list
for s in sub_df["Coordinates"]:
    try:
        polys = ast.literal_eval(s)
        assert isinstance(polys, list)
        # polys may be [] or list of polygons (lists/tuples)
    except Exception as e:
        raise ValueError(f"Bad Coordinates cell: {s[:80]}... -> {e}")

sub_df.to_csv("submissionH.csv", index=False)


# sanity checks to avoid "metric error"
assert sub_df.shape[0] == 1000, "Submission must have exactly 1000 rows."
assert sub_df["ImageID"].is_unique, "Duplicate ImageID rows."
assert sub_df["ImageID"].min() >= 0, "ImageID should start at 1."
assert sub_df["ImageID"].max() <= 1000, "ImageID should not exceed number of test images."

[10/07 00:09:48 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


In [62]:
#VISUALLLLLIZAÇÂAO"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""





VAL_IMG_DIR   = "/kaggle/working/processed/val/image"

import os, cv2, json, numpy as np, pandas as pd
from detectron2.engine import DefaultPredictor

# --- Build a predictor that uses your trained weights ---
cfg_pred = cfg.clone()
# Try to use the final trained checkpoint; fall back to last checkpoint; else keep the existing weights.
trained_path = os.path.join(cfg_pred.OUTPUT_DIR, "model_final.pth")
if not os.path.exists(trained_path):
    last_ckpt_file = os.path.join(cfg_pred.OUTPUT_DIR, "last_checkpoint")
    if os.path.exists(last_ckpt_file):
        with open(last_ckpt_file, "r") as f:
            ckpt_name = f.read().strip()
        maybe_path = os.path.join(cfg_pred.OUTPUT_DIR, ckpt_name)
        if os.path.exists(maybe_path):
            trained_path = maybe_path

if os.path.exists(trained_path):
    cfg_pred.MODEL.WEIGHTS = trained_path  # use trained weights
else:
    print(f"[Warning] Trained weights not found at {trained_path}. Using cfg.MODEL.WEIGHTS as-is.")

cfg_pred.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # adjust if needed
predictor = DefaultPredictor(cfg_pred)

# --- Take first 10 validation images (sorted) ---
VALID_EXTS = (".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp")
val_images = sorted([f for f in os.listdir(VAL_IMG_DIR) if f.lower().endswith(VALID_EXTS)])[:10]

# --- Output dirs & collectors ---
overlay_dir = "/kaggle/working/val_overlays"
os.makedirs(overlay_dir, exist_ok=True)
val_results = []  # to store polygons like your test loop

for img_name in val_images:
    img_path = os.path.join(VAL_IMG_DIR, img_name)
    im_bgr = cv2.imread(img_path)
    if im_bgr is None:
        print(f"[Skip] Could not read {img_path}")
        continue

    outputs = predictor(im_bgr)
    inst = outputs["instances"].to("cpu")

    masks = inst.pred_masks.numpy() if inst.has("pred_masks") else []
    # Build polygons like your test loop
    polygons = []
    for mask in masks:
        mask_uint8 = mask.astype("uint8")
        contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        for cnt in contours:
            pts = cnt.squeeze(1)
            if pts.size == 0:
                continue
            poly = [(int(x), int(y)) for x, y in pts]
            if poly and poly[0] != poly[-1]:
                poly.append(poly[0])  # close polygon
            polygons.append(poly)

    # Save polygons using same string format you used
    coord_str = str(polygons) if polygons else "[]"

    # Derive a numeric ID if filename is numeric; else keep the name
    base, _ = os.path.splitext(img_name)
    try:
        image_id = int(base)
    except ValueError:
        image_id = base

    val_results.append({"ImageID": image_id, "Coordinates": coord_str})

    # --- Make red overlay (fill + contour) ---
    overlay = im_bgr.copy()
    # Fill red on mask pixels (BGR -> red is (0,0,255))
    if len(masks) > 0:
        union_mask = np.any(masks, axis=0)
        overlay[union_mask] = (0, 0, 255)

    # Blend with original
    alpha = 0.5
    vis = cv2.addWeighted(overlay, alpha, im_bgr, 1 - alpha, 0)

    # Optional: red contour outlines for crisp edges
    for mask in masks:
        cnts, _ = cv2.findContours(mask.astype("uint8"), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(vis, cnts, -1, (0, 0, 255), 2)

    # Save visualization next to working dir
    out_path = os.path.join(overlay_dir, f"val_overlay_{img_name}")
    cv2.imwrite(out_path, vis)

# Save polygons for these 10 val images (handy for debugging)
val_df = pd.DataFrame(val_results)
val_df.to_csv("/kaggle/working/val_overlays/val_first10_footprintsB.csv", index=False)

print(f"Saved {len(val_images)} overlays to: {overlay_dir}")
print("Saved polygons to: /kaggle/working/val_first10_footprints.csv")


[10/06 20:22:51 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
Saved 10 overlays to: /kaggle/working/val_overlays
Saved polygons to: /kaggle/working/val_first10_footprints.csv
